MODAL WITH PYTORCH (LINK OF GITHUB)
https://github.com/modAL-python/modAL/blob/master/examples/pytorch_integration.py

IMPORT NECESSARY PACKAGES

In [1]:
import numpy as np
import torch 
from modAL.models import ActiveLearner
from skorch import NeuralNetClassifier
from torch import nn
from torchvision import transforms
from modAL.uncertainty import uncertainty_sampling
from PIL import Image


In [2]:
import sys
sys.path.insert(0, "..")

In [3]:
#device = "cuda" if torch.cuda.is_available() else "cpu"
device ="cpu"

LOAD THE TRAINING DATA

In [4]:
import pickle
with open('../dataset/cub_pickles/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
f.close()

with open('../dataset/cub_pickles/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
f.close()


LOAD THE TESTING DATA

In [5]:
with open('../dataset/cub_pickles/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
f.close()

with open('../dataset/cub_pickles/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)
f.close()

LOAD THE IMAGES, CONCEPTS AND TARGET LABELS FOR TRAINING DATASET

In [6]:
img_shape=32
resize = transforms.Resize( size=(img_shape, img_shape) ) #---- resize the image  
convert_tensor = transforms.ToTensor() #------- transform the image into tensor

X_list = []
concept_list = []
target_list = [] 
for i in range (len(X_train)):
    img = Image.open(X_train[i]['img_path'])
    img = convert_tensor(resize(img))
    img_np = img.numpy()
    X_list.append(img_np)
    concept_list.append(X_train[i]['attribute_label'])
    target_list.append(y_train[i])

X_train = torch.tensor(X_list).to(device)
concepts = torch.tensor(concept_list).to(device)
y_train = torch.tensor(target_list).to(device)


C:\Users\DELL\AppData\Local\Temp\ipykernel_1824\3006283836.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  X_train = torch.tensor(X_list).to(device)


In [7]:
from source.models import CBM_for_AL

model = CBM_for_AL()
classifier = NeuralNetClassifier(CBM_for_AL,
                                 max_epochs=100,
                                 criterion=nn.CrossEntropyLoss,
                                 optimizer=torch.optim.Adam,
                                 train_split=None,
                                 verbose=1,
                                 device=device)

In [8]:
# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)

X_initial = X_train[initial_idx]
concept_initial = concepts[initial_idx]
y_initial = y_train[initial_idx]


In [9]:
# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)
concept_pool = np.delete(concepts, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)



In [10]:

"""
Training the ActiveLearner
"""

# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=X_initial, y_training=y_initial,
    query_strategy=uncertainty_sampling
)

  epoch    train_loss     dur
-------  ------------  ------
      1        6.9119  2.2647
      2        5.2454  1.5160
      3        5.2379  1.5677
      4        5.2136  1.5148
      5        5.2124  1.4685
      6        5.2124  1.4781
      7        5.2118  1.5272
      8        5.2113  1.4325
      9        5.2109  1.5314
     10        5.2107  1.6357
     11        5.2106  1.4868
     12        5.2105  1.6305
     13        5.2103  1.6515
     14        5.2102  1.5301
     15        5.2101  1.4672
     16        5.2100  1.4627
     17        5.2099  1.4523
     18        5.2098  1.4204
     19        5.2097  1.4234
     20        5.2096  1.4325
     21        5.2096  1.4512
     22        5.2095  1.4333
     23        5.2094  1.4181
     24        5.2093  1.4072
     25        5.2093  1.4176
     26        5.2092  1.4440
     27        5.2091  1.4851
     28        5.2090  1.3920
     29        5.2090  1.4298
     30        5.2089  1.4410
     31        5.2089  1.4079
     32   

In [11]:




# the active learning loop
n_queries = 10
for idx in range(n_queries):
    query_idx, query_instance = learner.query(X_pool, n_instances=100)
    
    learner.teach(X_pool[query_idx], y_pool[query_idx], only_new=True)
    # remove queried instance from pool
    X_pool = np.delete(X_pool, query_idx, axis=0)
    y_pool = np.delete(y_pool, query_idx, axis=0)



Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        5.3054  0.1693
      2        4.9500  0.1614
      3        4.6382  0.1722
      4        4.5497  0.1667
      5        4.6041  0.1660
      6        4.3851  0.1551
      7        4.3804  0.1488
      8        4.3554  0.1600
      9        4.3345  0.1435
     10        4.3008  0.1546
     11        4.2276  0.1604
     12        4.1510  0.1611
     13        4.0999  0.1557
     14        3.9886  0.1482
     15        3.8761  0.1496
     16        3.7417  0.1598
     17        3.5685  0.1553
     18        4.0868  0.1606
     19        3.4955  0.1609
     20        3.7646  0.1686
     21        3.4298  0.1920
     22        3.1744  0.1901
     23        3.2402  0.1997
     24        2.8323  0.1974
     25        2.7566  0.1937
     26        2.5662  0.1986
     27        2.3343  0.1922
     28        2.2776  0.1917
     29        2.0953  0.1761
     30        1.91

-------- IN THIS PART, WE TRAIN THE MODEL FROM SCRATCH USING THE NEWLY ADDED DATA FROM POOL ------

In [12]:
from source.models import CBM_for_AL


classifier_2 = NeuralNetClassifier(CBM_for_AL,
                                 max_epochs=100,
                                 criterion=nn.CrossEntropyLoss,
                                 optimizer=torch.optim.Adam,
                                 train_split=None,
                                 verbose=1,
                                 device=device)

In [13]:
# assemble initial data
n_initial = 1000
initial_idx = np.random.choice(range(len(X_train)), size=n_initial, replace=False)
idx_all =np.array(initial_idx, copy=True)

X_initial = X_train[initial_idx]
concept_initial = concepts[initial_idx]
y_initial = y_train[initial_idx]


In [14]:
# generate the pool
# remove the initial data from the training dataset
X_pool = np.delete(X_train, initial_idx, axis=0)
concept_pool = np.delete(concepts, initial_idx, axis=0)
y_pool = np.delete(y_train, initial_idx, axis=0)



In [15]:

"""
Training the ActiveLearner
"""

# initialize ActiveLearner
learner_2 = ActiveLearner(
    estimator=classifier_2,
    X_training=X_initial, y_training=y_initial,
    query_strategy=uncertainty_sampling
)

  epoch    train_loss     dur
-------  ------------  ------
      1        5.5875  1.6730
      2        5.2688  1.5968
      3        5.2249  1.5953
      4        5.2147  1.5915
      5        5.2020  1.5607
      6        5.1871  1.5919
      7        5.1534  1.5530
      8        5.1091  1.5665
      9        5.0312  1.6447
     10        4.9601  1.5847
     11        4.8665  1.5654
     12        4.8113  1.6049
     13        4.7470  1.5897
     14        4.6383  1.5920
     15        4.4950  1.5812
     16        4.3576  1.6048
     17        4.1680  1.6209
     18        4.1414  1.6395
     19        4.1940  1.5996
     20        4.0051  1.5943
     21        3.8854  1.6198
     22        3.9427  1.6005
     23        4.2099  1.5710
     24        3.9665  1.6010
     25        3.6346  1.5894
     26        3.4146  1.5859
     27        3.1591  1.5786
     28        2.9858  1.5750
     29        2.8794  1.6506
     30        2.7808  1.5723
     31        2.8698  1.5419
     32   

In [17]:
# the active learning loop
n_queries = 10
for idx in range(n_queries):
    query_idx, query_instance = learner_2.query(X_pool, n_instances=200)
    idx_all= np.append(idx_all, query_idx)
    X_new = X_train[idx_all]
    y_new = y_train[idx_all]
    learner_2.fit(X_new, y_new)
    # remove queried instance from pool
    X_pool = np.delete(X_train, query_idx, axis=0)
    y_pool = np.delete(y_train, query_idx, axis=0)



Re-initializing module.
Re-initializing optimizer.
  epoch    train_loss     dur
-------  ------------  ------
      1        7.3890  1.7541
      2        5.2549  1.7319
      3        5.2365  1.7438
      4        5.2338  1.8284
      5        5.2353  1.7433
      6        5.2340  1.7511
      7        5.2335  1.7277
      8        5.2335  1.7304
      9        5.2333  1.7403
     10        5.2329  1.7512
     11        5.2326  1.7614
     12        5.2325  1.7416
     13        5.2324  1.7393
     14        5.2322  1.7530
     15        5.2321  1.7631
     16        5.2320  1.8775
     17        5.2319  1.7178
     18        5.2318  1.6931
     19        5.2317  1.6901
     20        5.2316  1.6775
     21        5.2315  1.7550
     22        5.2314  1.7808
     23        5.2313  1.7450
     24        5.2313  1.7295
     25        5.2312  1.7818
     26        5.2311  1.8102
     27        5.2310  1.7249
     28        5.2310  1.7255
     29        5.2309  1.7136
     30        5.23